<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>XGBoost</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Catboost</a></span></li><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Combine-RF,-CB-and-GB" data-toc-modified-id="Combine-RF,-CB-and-GB-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Combine RF, CB and GB</a></span></li><li><span><a href="#Plot-AUC-Curve" data-toc-modified-id="Plot-AUC-Curve-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plot AUC Curve</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Cat-Boost" data-toc-modified-id="Cat-Boost-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Cat Boost</a></span></li><li><span><a href="#Random-Forest,-Cat-Boost,-Gradient-Boost" data-toc-modified-id="Random-Forest,-Cat-Boost,-Gradient-Boost-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Random Forest, Cat Boost, Gradient Boost</a></span></li></ul></div>

# Split into train and test data

In [2]:
import pandas as pd
bids_df_combined = pd.read_csv('combined2.csv')

In [64]:
train_df = bids_df_combined.loc[bids_df_combined.outcome != 'Unknown']
test_df = bids_df_combined.loc[bids_df_combined.outcome == 'Unknown']
print(f'Length of train_df: {len(train_df)} / 2013')
print(f'Length of test_df: {len(test_df)} / 4700')

Length of train_df: 2013 / 2013
Length of test_df: 4700 / 4700


In [4]:
bids_df_combined.columns

Index(['bidder_id', 'payment_account', 'address', 'outcome',
       'min_bids_per_auction', 'max_bids_per_auction', 'mean_bids_per_auction',
       'median_bids_per_auction', 'total_bids', 'num_of_auctions',
       'min_bids_per_url', 'max_bids_per_url', 'mean_bids_per_url',
       'median_bids_per_url', 'num_of_url_used', 'min_bids_per_ip',
       'max_bids_per_ip', 'mean_bids_per_ip', 'median_bids_per_ip',
       'num_of_ip_used', 'bids_in_set1', 'bids_in_set2', 'bids_in_set3',
       'bids_in_first3', 'bids_in_last3', 'mean_bid_per_auction_during_peak',
       'mean_time_between_bids', 'median_time_between_bids', 'num_of_merch',
       'unique_devices_used', 'unique_countries_used', 'ip_used_by_bot',
       'fraction_of_ips_used_by_bot'],
      dtype='object')

In [62]:
# removing the 5 bots with only one total_bid
train_df = train_df.loc[(train_df.outcome!='1.0') | (train_df.total_bids!=1)]

# Implement Model Here

In [37]:
features = ['min_bids_per_auction', 'max_bids_per_auction', 'mean_bids_per_auction',
       'median_bids_per_auction', 'total_bids', 'num_of_auctions',
       'min_bids_per_url', 'max_bids_per_url', 'mean_bids_per_url',
       'median_bids_per_url', 'num_of_url_used', 'min_bids_per_ip',
       'max_bids_per_ip', 'mean_bids_per_ip', 'median_bids_per_ip',
       'num_of_ip_used', 'bids_in_set1', 'bids_in_set2', 'bids_in_set3','bids_in_first3','bids_in_last3',
           'mean_bid_per_auction_during_peak','mean_time_between_bids','median_time_between_bids']
X = train_df[features]
y = train_df['outcome'].apply(float)

##### XGBoost

In [40]:
import statistics
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import KFold
#from sklearn.model_selection import train_test_split

kf = KFold(n_splits=100)

model = xgb.XGBClassifier(random_state=5)

k_fold_Accuracy = []

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    xgb_model = model.fit(X_train,y_train)
    y_pred = xgb_model.predict(X_test)
    
    accuracy = metrics.accuracy_score(y_test, y_pred)
    k_fold_Accuracy.append(accuracy)
    
print('The Accuracy for each fold is:', k_fold_Accuracy)
print('The average accuracy is:', statistics.mean(k_fold_Accuracy)) 
# 0.9590714285714286

/Users/cherylperyl/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:09:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[05:09:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[05:09:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[05:09:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:09:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

##### Random Forest

In [39]:
from sklearn.ensemble import RandomForestClassifier

kf = KFold(n_splits=100)

model = RandomForestClassifier(random_state=5, n_estimators=225)

k_fold_Accuracy = []

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    rfc_model = model.fit(X_train,y_train)
    y_pred = rfc_model.predict(X_test)
    
    accuracy = metrics.accuracy_score(y_test, y_pred)
    k_fold_Accuracy.append(accuracy)

print('The Accuracy for each fold is:', k_fold_Accuracy)
print('The average accuracy is:', statistics.mean(k_fold_Accuracy)) 

# 0.9605714285714285

The Accuracy for each fold is: [1.0, 1.0, 1.0, 0.9523809523809523, 1.0, 1.0, 0.9523809523809523, 0.9523809523809523, 1.0, 1.0, 0.9, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 0.9, 0.95, 0.95, 0.95, 0.95, 0.9, 0.95, 0.95, 0.95, 0.95, 1.0, 0.9, 0.95, 1.0, 0.95, 0.95, 0.95, 0.9, 0.95, 0.95, 0.95, 0.9, 1.0, 0.9, 0.85, 1.0, 1.0, 1.0, 0.9, 0.95, 0.95, 0.85, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 0.9, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 0.9, 0.85, 0.9, 1.0, 1.0, 0.95, 0.8, 0.95, 0.9, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 0.9]
The average accuracy is: 0.9605714285714285


##### Catboost

In [42]:
from catboost import CatBoostClassifier

kf = KFold(n_splits=100)

model = CatBoostClassifier(random_seed=5)

k_fold_Accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    cbc_model = model.fit(X_train,y_train,verbose=False)
    y_pred = cbc_model.predict(X_test)

    accuracy = metrics.accuracy_score(y_test, y_pred)
    k_fold_Accuracy.append(accuracy)
    
print('The Accuracy for each fold is:', k_fold_Accuracy)
print('The average accuracy is:', statistics.mean(k_fold_Accuracy))
# 0.9635714285714285

The Accuracy for each fold is: [1.0, 1.0, 1.0, 0.9523809523809523, 1.0, 1.0, 0.9523809523809523, 0.9523809523809523, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 1.0, 0.85, 0.9, 0.95, 1.0, 0.95, 0.9, 0.95, 0.95, 0.95, 0.95, 1.0, 0.9, 0.95, 1.0, 0.95, 0.95, 0.95, 0.9, 0.95, 1.0, 0.95, 0.9, 1.0, 0.9, 0.9, 1.0, 1.0, 1.0, 0.9, 0.95, 0.95, 0.85, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 0.85, 0.85, 1.0, 1.0, 1.0, 0.95, 0.85, 1.0, 0.9, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 0.9]
The average accuracy is: 0.9635714285714285


In [ ]:
# for tuning parameters

In [262]:
from catboost import CatBoostClassifier

kf = KFold(n_splits=10)

depth=[i for i in range(10)] #3,7

for d in depth:
    model = CatBoostClassifier(random_seed=5, iterations=1000,depth=3,l2_leaf_reg=d)

    k_fold_Accuracy = []

    for train_index, test_index in kf.split(X):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        cbc_model = model.fit(X_train,y_train,verbose=False)
        y_pred = cbc_model.predict(X_test)

        accuracy = metrics.accuracy_score(y_test, y_pred)
        k_fold_Accuracy.append(accuracy)

    #print('The Accuracy for each fold is:', k_fold_Accuracy)
    print('The average accuracy is:', statistics.mean(k_fold_Accuracy), d)
# previous: 0.9577557755775578
# latest:   0.9567632136347963
# latest2:  0.9577607014432786
# latest3:  0.9547780897492735

The average accuracy is: 0.9557780404906162 0
The average accuracy is: 0.9537929166050934 1
The average accuracy is: 0.9532904783015616 2
The average accuracy is: 0.952302842224521 3
The average accuracy is: 0.9542879661100439 4
The average accuracy is: 0.9532904783015614 5
The average accuracy is: 0.9547854785478548 6
The average accuracy is: 0.9547879414807152 7
The average accuracy is: 0.9552780651199448 8
The average accuracy is: 0.9542855031771834 9


##### Logistic Regression

In [44]:
# Train Test Split
from sklearn.linear_model import LogisticRegression

kf = KFold(n_splits=10)

log_reg = LogisticRegression(max_iter=300)

k_fold_accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    log_reg_model = log_reg.fit(X_train, y_train)
    y_pred = log_reg_model.predict(X_test)

    accuracy = metrics.accuracy_score(y_test, y_pred)
    k_fold_accuracy.append(accuracy)
    
print('The accuracy for each fold is:', k_fold_accuracy)
print('The average accuracy is:', statistics.mean(k_fold_accuracy))
# previous: 0.9408625190877297
# latest:   0.943342692478203
# latest2:  0.9438377419831535

##### Combine RF, CB and GB

In [45]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import statistics

kf = KFold(n_splits=100)
model1 = RandomForestClassifier(random_state=5, n_estimators=225)
model2 = GradientBoostingClassifier(learning_rate=0.01,random_state=5) 
model3 = CatBoostClassifier(random_seed=5)

k_fold_accuracy = []
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model1.fit(X_train,y_train)
    model2.fit(X_train,y_train)
    model3.fit(X_train,y_train,verbose=False)
    
    pred1 = model1.predict(X_test)
    pred2 = model2.predict(X_test)
    pred3 = model3.predict(X_test)
    
    final_pred = []
    for i in range(0,len(X_test)):
        final_pred.append(statistics.mode([pred1[i],pred2[i],pred3[i]]))

    accuracy = metrics.accuracy_score(y_test, final_pred)
    k_fold_accuracy.append(accuracy)
    
print('The accuracy for each fold is:', k_fold_accuracy)
print('The average accuracy is:', statistics.mean(k_fold_accuracy)) 

# Record latest results here
# 0.9605714285714285 used mode

The accuracy for each fold is: [1.0, 1.0, 1.0, 0.9523809523809523, 1.0, 1.0, 0.9523809523809523, 0.9523809523809523, 1.0, 1.0, 0.9, 0.95, 1.0, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 1.0, 0.95, 0.9, 0.9, 0.95, 0.95, 0.95, 0.9, 0.95, 0.95, 0.95, 0.95, 1.0, 0.9, 0.95, 1.0, 0.95, 0.95, 0.95, 0.9, 0.95, 0.95, 0.95, 0.9, 1.0, 0.9, 0.85, 1.0, 1.0, 1.0, 0.9, 0.95, 0.95, 0.85, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 0.9, 1.0, 0.95, 1.0, 0.95, 1.0, 1.0, 1.0, 0.95, 0.95, 1.0, 1.0, 1.0, 0.95, 0.9, 0.85, 0.9, 1.0, 1.0, 0.95, 0.8, 0.95, 0.9, 1.0, 1.0, 1.0, 1.0, 0.95, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 0.9]
The average accuracy is: 0.9600714285714286


##### Plot AUC Curve

In [63]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn import metrics
import statistics
from sklearn.metrics import roc_curve, auc, roc_auc_score

n_estimators = [i for i in range(99, 401)]
train_results = []
test_results = []
for estimator in n_estimators:
    rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
    rf.fit(X_train, y_train)
    train_pred = rf.predict(X_train)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    train_results.append(roc_auc)
    y_pred = rf.predict(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_results.append(roc_auc)
    
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(n_estimators, train_results, 'b', label='Train AUC')
line2, = plt.plot(n_estimators, test_results, 'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [20, 4700]

# FINAL MODEL

In [50]:
# edit features here
features = ['min_bids_per_auction', 'max_bids_per_auction', 'mean_bids_per_auction',
       'median_bids_per_auction', 'total_bids', 'num_of_auctions',
       'min_bids_per_url', 'max_bids_per_url', 'mean_bids_per_url',
       'median_bids_per_url', 'num_of_url_used', 'min_bids_per_ip',
       'max_bids_per_ip', 'mean_bids_per_ip', 'median_bids_per_ip',
       'num_of_ip_used', 'bids_in_set1', 'bids_in_set2', 'bids_in_set3','bids_in_first3','bids_in_last3',
           'mean_bid_per_auction_during_peak','mean_time_between_bids','median_time_between_bids']

# do not edit this part
X_train = train_df[features]
y_train = train_df['outcome'].apply(float)
X_test = test_df[features]

In [235]:
# implement model below

##### Random Forest

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import statistics

# implement model here
model = RandomForestClassifier(random_state=5, n_estimators=225)

model.fit(X_train, y_train)
print('Number of Trees used : ', model.n_estimators) # Find out number of trees used. 100 is a default

y_pred_proba = model.predict_proba(X_test)
final_pred_proba = list(y_pred_proba[:, 1])
print('- final_pred_proba ready for submission -')

Number of Trees used :  225
- final_pred_proba ready for submission -


##### Cat Boost

In [59]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(random_seed=5,iterations=1000,depth=7,l2_leaf_reg=0)

model.fit(X_train, y_train, verbose=False)  # set verbose=False if you find the logs too long

y_pred = model.predict(X_test)

y_pred_proba = model.predict_proba(X_test)
final_pred_proba = list(y_pred_proba[:, 1])
print('- final_pred_proba ready for submission -')

- final_pred_proba ready for submission -


##### Random Forest, Cat Boost, Gradient Boost

In [51]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn import metrics
import statistics

model1 = RandomForestClassifier(random_state=5, n_estimators=225)
model2 = GradientBoostingClassifier(learning_rate=0.01,random_state=5) 
model3 = CatBoostClassifier(random_seed=5)

model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train,verbose=False)
    
pred_proba1 = list((model1.predict_proba(X_test))[:, 1])
pred_proba2 = list((model2.predict_proba(X_test))[:, 1])
pred_proba3 = list((model3.predict_proba(X_test))[:, 1])
    
final_pred_proba = []
for i in range(len(X_test)):
    final_pred_proba.append(np.mean([pred_proba1[i], pred_proba2[i], pred_proba3[i]]))

print('- final_pred_proba ready for submission -')

In [52]:
len(final_pred_proba)

4700

# FORMING THE CSV FILE

before forming the csv file, ensure that you `.predict_proba` on your model into a var named `y_pred_proba`

In [45]:
# run these lines first to form list of predicted probabilities
# one_model
y_pred_proba_formatted = list(y_pred_proba[:, 1])
#print(y_pred_proba_formatted)

In [53]:
bidder_id_list = list(test_df.bidder_id) #4.7 bidders
predictions_list = final_pred_proba #predict proba list

pred_list = [['Id','Predicted']]
for i in range(len(predictions_list)):
    pred_list.append([bidder_id_list[i],predictions_list[i]])
df = pd.DataFrame(pred_list[1:], columns = pred_list[0])

df.to_csv('RF_CB_GB_avg_emsemble2.csv', index=False) # change file name everytime you have a new set of predictions

In [54]:
# ensure there are 4,700 rows before submission
df

,Id,Predicted
0,0013f9b8d1f462df4462e1c1216e441ba6e18,0.007053
1,0014e3b911d1420e43ced1dc4fc18fde0fd0c,0.006954
2,002828f800c5132e297cfe3d44fbde9aeac51,0.006992
3,0040cea6b93afd86768c365d89513ffb7c0ba,0.007323
4,00512db4ae953baed983a4bcfa335e7412013,0.038122
...,...,...
4695,ffd49be672b1ba493b07dccf29311045c5392,0.056802
4696,ffdd8ed91a683b0f2a8237798ebe9214e3c43,0.007188
4697,ffde8717e9a547d536a4e9c3f44782829c021,0.007273
4698,ffe66dcb4b34bdbb5e17d7db7b1395e3fbbb7,0.013193
